In [1]:
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
from transformers import pipeline

In [2]:
model_name = "facebook/mbart-large-50"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [3]:
dataset = load_dataset("ai4bharat/samanantar", "mr", split="train[:2%]")

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/244M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/244M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3627480 [00:00<?, ? examples/s]

In [4]:
tokenizer.src_lang = "en_XX"
tokenizer.tgt_lang = "mr_IN"

def preprocess(examples):
    inputs = examples['src']
    targets = examples['tgt']
    model_inputs = tokenizer(inputs, max_length=128, truncation=True,padding='max_length')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True,padding='max_length')
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [5]:
tokenized = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/72550 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [6]:
seq2seq_collator = DataCollatorForSeq2Seq(tokenizer, model=model,padding='longest',return_tensors='pt')

In [7]:
args = TrainingArguments(
    output_dir="./mbart-en-mr",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=1,
    fp16=True
)

In [8]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized,
    tokenizer=tokenizer,
    data_collator=seq2seq_collator
)

/tmp/ipython-input-4122455342.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: khetres3636 (khetres3636-na) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,1.604600
1000,0.487000
1500,0.455500
2000,0.441600
2500,0.437900
3000,0.415300
3500,0.412600
4000,0.408200
4500,0.404200
5000,0.398700


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3917: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=9069, training_loss=0.47308708049586146, metrics={'train_runtime': 9908.0587, 'train_samples_per_second': 7.322, 'train_steps_per_second': 0.915, 'total_flos': 1.96531580829696e+16, 'train_loss': 0.47308708049586146, 'epoch': 1.0})

In [10]:
model.save_pretrained("(en-mar)_model")
tokenizer.save_pretrained("(en-mar)_tokenizer")

('(en-mar)_tokenizer/tokenizer_config.json',
 '(en-mar)_tokenizer/special_tokens_map.json',
 '(en-mar)_tokenizer/sentencepiece.bpe.model',
 '(en-mar)_tokenizer/added_tokens.json',
 '(en-mar)_tokenizer/tokenizer.json')

In [15]:
translator = pipeline(
    "translation",
    model=model,
    tokenizer=tokenizer,
    src_lang="en_XX",   # source language ID (English)
    tgt_lang="mr_IN"    # target language ID (Marathi)
)

Device set to use cuda:0


In [62]:
text="good morning , i am exiceted for today"

In [65]:
test_sentences = [
    "How are you today?",
    "I am learning artificial intelligence.",
    "This is a beautiful place.",
    "The weather is very hot today.",
    "Machine learning is a part of artificial intelligence."
]

In [66]:
result = translator(test_sentences, max_length=100, min_length=1, do_sample=False)

In [67]:
result

[{'translation_text': 'तुम्ही आज कसे आहेस?'},
 {'translation_text': 'मी कृत्रिम बुद्धिमत्ता शिकत आहे.'},
 {'translation_text': 'हे एक सुंदर ठिकाण आहे.'},
 {'translation_text': 'आज सकाळचे वातावरण खूपच थंड आहे.'},
 {'translation_text': 'कृत्रिम बुद्धिमत्ता हा मानवी बुद्धिमत्ताचा भाग आहे.'}]